In [ ]:
# %pip install langchain-openai python-dotenv
%pip install langchain_pinecone

In [1]:
from dotenv import load_dotenv

# 환경변수 불러옴
load_dotenv()

True

In [2]:
from langchain_upstage import ChatUpstage

chat = ChatUpstage()

In [ ]:
# response = chat.invoke("한국의 수도는?")
# print(response)

content='한국의 수도는 서울입니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 15, 'total_tokens': 23, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'solar-mini-240612', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-5a57535f-7140-42b4-90b6-76ddfc31b52f-0' usage_metadata={'input_tokens': 15, 'output_tokens': 8, 'total_tokens': 23, 'input_token_details': {}, 'output_token_details': {}}


In [3]:
# 문서 로드
from langchain_community.document_loaders import Docx2txtLoader, TextLoader

# loader = Docx2txtLoader("./tax.docx")
loader = TextLoader("./docs/dorm.md")
data = loader.load()
data

[Document(metadata={'source': './docs/dorm.md'}, page_content='# SK 하이닉스 이천 기숙사\n\n## 소개\n\n이천 기숙사는 경기도 이천시 부발읍 일대 44만 평 부지에 4개 Site, 17개동, 3,200여 개 호실을 갖추고 여러분을 기다리고 있습니다.\n기숙사와 함께 다양한 커뮤니티 시설이 제공되며 라운지, 카페, 헬스장, 문화 강좌실 등 생활지원시설 및 문화시설에서 입숙인들이 안전하고 편안한 시간을 즐길 수 있도록 다양한 지원과 서비스를 준비하고 있습니다.\n\n## 기숙사동\n\n1. 행복1마을\n\n   - 주소: 경기도 이천시 대산로 288번길, 89\n\n2. 행복2마을\n\n   - 주소: 경기도 이천시 대월면 대산로 288번길, 48\n\n3. 청운기숙사\n\n   - 주소: 경기도 이천시 부발읍 경충대로 2091 SK하이닉스 청운관\n\n4. 현우기숙사\n\n   - 주소: 경기도 이천시 부발읍 경충대로 2041번길, 54\n\n## 기숙사 홈페이지\n\n- 홈페이지 이름: 하이홈(Hihome)\n- 주소: https://hihome.skhynix.com/main\n- 설명: 구성원의 생활/지원 서비스(입·퇴실, 이동, 하자 등)는 더욱 빠르게 해결하고, 주거 시설 Infra (Cafétreia, 주차, 택배 등) 및 문화/편의시설 운영 정보도 실시간으로 제공해 더욱 편리한 생활을 지원합니다\n\n## 관리지원 센터 및 시설 연락처와 운영정보\n\n- 현우기숙사 생활지원센터\n\n* 연락처: 031-5185-3420\n* 위치: 경기도 이천시 부발읍 경중대로 2041번길 54 현우기숙사\n* 운영시간: 평일 08:30~17:30 / 유연근무\n\n- 행복마을 시설관리\n\n* 연락처: 031-5185-3883\n* 위치: [ 행복 1마을 ]경기도 이천시 대산로 288번길, 89 [ 행복 2마을 ] 경기도 이천시 대산로 288번길, 48\n* 운영시간: 평일 08:30~17:30 / 유연근무

In [5]:
# 문서 쪼개기
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1500,
    chunk_overlap=200,
)

document_list = loader.load_and_split(text_splitter=text_splitter)

In [ ]:
# 임베딩
from langchain_upstage import UpstageEmbeddings


embedding = UpstageEmbeddings(
    model="solar-embedding-1-large"
)

In [8]:
from langchain_pinecone import PineconeVectorStore

# 데이터를 처음 저장할 때 
index_name = 'dorm-upstage-index'
database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

In [9]:
query = '택배 센터는 몇 시까지?'

# `k` 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
retrieved_docs = database.similarity_search(query, k=3)

In [10]:
retrieved_docs[0]

Document(id='a3a14706-35cf-4177-9984-993206e1b33b', metadata={'source': './dorm.md'}, page_content='- 행복 1마을 경비실\n\n* 연락처: 031-633-1968\n* 위치: 경기도 이천시 대산로 288번길, 89 (행복1마을)\n* 운영시간: 24시간\n\n- 행복 2마을 경비실\n\n* 연락처: 031-5185-4073\n* 위치: 경기도 이천시 대산로 288번길, 48 (행복2마을)\n* 운영시간: 24시간\n\n- 청운 기숙사 1관 경비실\n\n* 연락처: 010-3294-4231\n* 위치: 경기도 이천시 부발읍 경충대로 2091 SK하이닉스 (청운관)\n* 운영시간: 24시간\n\n- 청운 기숙사 2관 경비실\n\n* 연락처: 010-3453-4231\n* 위치: 경기도 이천시 부발읍 경충대로 2091 SK하이닉스 (청운관)\n* 운영시간: 24시간\n\n- 청운 기숙사 3관 경비실\n\n* 연락처: 010-3428-4231\n* 위치: 경기도 이천시 부발읍 경충대로 2091 SK하이닉스 (청운관)\n* 운영시간: 24시간\n\n- 청운 기숙사 4관 경비실\n\n* 연락처: 010-3296-4231\n* 위치: 경기도 이천시 부발읍 경충대로 2091 SK하이닉스 (청운관)\n* 운영시간: 24시간\n\n- 청운 기숙사 5관 경비실\n\n* 연락처: 010-3295-4231\n* 위치: 경기도 이천시 부발읍 경충대로 2091 SK하이닉스 (청운관)\n* 운영시간: 24시간\n\n- 청운 기숙사 6관 경비실\n\n* 연락처: 010-3462-4231\n* 위치: 경기도 이천시 부발읍 경충대로 2091 SK하이닉스 (청운관)\n* 운영시간: 24시간\n\n- 택배센터\n\n* 연락처:\n  - 031-633-2306\n  - 031-694-3407\n  - 031-8094-7828\n* 위치:\n  - 행복1마을 복지A동 1층\n  - 행복2마을 커뮤니티센터 2층\n  - 현

In [11]:

from langchain_upstage import ChatUpstage

llm = ChatUpstage()

In [ ]:
from langchain import hub

# prompt = hub.pull("rlm/rag-prompt")
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

index_name = 'dorm-upstage-index'
database = PineconeVectorStore(index_name=index_name, embedding=embedding)
retrieval_qa_chat_prompt = hub.pull("teddynote/rag-prompt-korean")

combine_docs_chain = create_stuff_documents_chain(llm, retrieval_qa_chat_prompt)
rag_chain = create_retrieval_chain(database.as_retriever(), combine_docs_chain)

query = '택배 센터는 몇 시까지?'

ai_message = rag_chain.invoke({"question": query, "input": query})


/Users/boosj/.pyenv/versions/3.10.15/envs/llm/lib/python3.10/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [18]:
ai_message

{'question': '택배 센터는 몇 시까지?',
 'input': '택배 센터는 몇 시까지?',
 'context': [Document(id='a3a14706-35cf-4177-9984-993206e1b33b', metadata={'source': './dorm.md'}, page_content='- 행복 1마을 경비실\n\n* 연락처: 031-633-1968\n* 위치: 경기도 이천시 대산로 288번길, 89 (행복1마을)\n* 운영시간: 24시간\n\n- 행복 2마을 경비실\n\n* 연락처: 031-5185-4073\n* 위치: 경기도 이천시 대산로 288번길, 48 (행복2마을)\n* 운영시간: 24시간\n\n- 청운 기숙사 1관 경비실\n\n* 연락처: 010-3294-4231\n* 위치: 경기도 이천시 부발읍 경충대로 2091 SK하이닉스 (청운관)\n* 운영시간: 24시간\n\n- 청운 기숙사 2관 경비실\n\n* 연락처: 010-3453-4231\n* 위치: 경기도 이천시 부발읍 경충대로 2091 SK하이닉스 (청운관)\n* 운영시간: 24시간\n\n- 청운 기숙사 3관 경비실\n\n* 연락처: 010-3428-4231\n* 위치: 경기도 이천시 부발읍 경충대로 2091 SK하이닉스 (청운관)\n* 운영시간: 24시간\n\n- 청운 기숙사 4관 경비실\n\n* 연락처: 010-3296-4231\n* 위치: 경기도 이천시 부발읍 경충대로 2091 SK하이닉스 (청운관)\n* 운영시간: 24시간\n\n- 청운 기숙사 5관 경비실\n\n* 연락처: 010-3295-4231\n* 위치: 경기도 이천시 부발읍 경충대로 2091 SK하이닉스 (청운관)\n* 운영시간: 24시간\n\n- 청운 기숙사 6관 경비실\n\n* 연락처: 010-3462-4231\n* 위치: 경기도 이천시 부발읍 경충대로 2091 SK하이닉스 (청운관)\n* 운영시간: 24시간\n\n- 택배센터\n\n* 연락처:\n  - 031-633-2306\n  - 031-694-3407\n 

In [26]:

from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [ ]:
# Langchain 권장은 아래 셀 방법으로 사용
# ai_message = qa_chain({"query": query})


/var/folders/99/8v3n2jl97gd7jk_s3lgkd4qm0000gn/T/ipykernel_6773/314547319.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ai_message = qa_chain({"query": query})


In [ ]:
# Langchain 권장 문법
ai_message = qa_chain.invoke({"query": query})


In [29]:
ai_message


{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 직장인의 소득세는 약 142만원입니다.'}